In [ ]:
import pandas as pd
import numpy as np

# ====== 경로 =======
CLEAN_DIR = "clean_vote_ver2"
DUMP_DIR = "dump_vote_ver2"

# ====== 전처리 =======
school   = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_school.csv")
user     = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_user.csv")
pay      = pd.read_csv(f"{CLEAN_DIR}/processed_accounts_paymenthistory.csv")
uqr      = pd.read_csv(f"{CLEAN_DIR}/processed_userquestionrecord.csv")
hackle   = pd.read_csv(f"{CLEAN_DIR}/processed_hackle_merge.csv")  # 리텐션용

# ====== dump =======
group = pd.read_csv(f"{DUMP_DIR}/accounts_group.csv")

In [43]:
# 1. 노출 계산

print("1단계: 노출")

# user_id str 변환
user['user_id'] = user['user_id'].astype(str)

# 조건 1: 학교 40명 이상
# user → group → school
active_schools = school[school['is_active_school'] == True]['id']
active_school_groups = group[group['school_id'].isin(active_schools)]['id']
user_in_active = user[user['group_id'].isin(active_school_groups)]
print(f"활성 학교 소속 유저: {len(user_in_active):,}")

# 조건 2: 학급 4명 이상
group_counts = user.groupby('group_id')['user_id'].nunique().reset_index()
group_counts.columns = ['group_id', 'count']
active_groups = group_counts[group_counts['count'] >= 4]['group_id']
active_group_users = user[user['group_id'].isin(active_groups)]
print(f"활성 학급: {len(active_groups):,}개 학급 / {len(active_group_users):,}명 유저")

# 노출 = A AND B
exposure_df = user_in_active[user_in_active['group_id'].isin(active_groups)]
exposure_users = set(exposure_df['user_id'].unique())

print(f"노출 유저 수 (A∩B): {len(exposure_users):,}명")
print(f"조건A만: {len(user_in_active):,}명")
print(f"조건B만: {len(active_group_users):,}명")
print(f"A∩B: {len(exposure_users):,}명")

1단계: 노출
활성 학교 소속 유저: 655,353
활성 학급: 56,957개 학급 / 636,487명 유저
노출 유저 수 (A∩B): 626,237명
조건A만: 655,353명
조건B만: 636,487명
A∩B: 626,237명


In [42]:
# 2. 유입 계산

print("2단계: 유입")

# 전체 가입 유저
inflow_all = set(user['user_id'].unique())
print(f"전체 가입 유저: {len(inflow_all):,}")

# 노출 → 유입
inflow_users = exposure_users & inflow_all
print(f"노출→유입 유저: {len(inflow_users):,}")

2단계: 유입
전체 가입 유저: 676,978
노출→유입 유저: 626,237


In [48]:
# 3. 참여 계산

print("3단계: 참여")

# user_id str 변환
uqr['user_id'] = uqr['user_id'].astype(str)
uqr['chosen_user_id'] = uqr['chosen_user_id'].astype(str)

# 투표한 유저 + 투표받은 유저
voted = set(uqr['user_id'].dropna().unique())
chosen = set(uqr['chosen_user_id'].dropna().unique())
vote_activity = voted | chosen
print(f"투표 활동 유저: {len(vote_activity):,}")

# 유입 중 참여
participation_users = inflow_users & vote_activity
print(f"참여 유저 수: {len(participation_users):,}")

3단계: 참여
투표 활동 유저: 15,452
참여 유저 수: 14,991


In [50]:
# 4. 리텐션 계산

print("4단계: 리텐션")

# 해클 이벤트 로그에서 활동 유저 추출
# str → float → int → str 순서로 변환 (.0 제거)
hackle['user_id'] = pd.to_numeric(hackle['user_id'], errors='coerce')  # 문자열이든 숫자든 float으로
hackle['user_id'] = hackle['user_id'].fillna(0).astype(int).astype(str)
# 0으로 변환된 것들 제거
hackle = hackle[hackle['user_id'] != '0']

active_users = set(hackle['user_id'].dropna().unique())
print(f"Hackle 이벤트 로그 있는 유저: {len(active_users):,}명")

# 참여 중 리텐션
retention_users = participation_users & active_users
print(f"리텐션 유저 수 (참여 Hackle 합집합): {len(retention_users):,}명")

4단계: 리텐션
Hackle 이벤트 로그 있는 유저: 226,365명
리텐션 유저 수 (참여 Hackle 합집합): 4,083명


In [51]:
# 5. 수익 계산

print("5단계: 수익")

pay['user_id'] = pay['user_id'].astype(str)
paid_users = set(pay['user_id'].dropna().unique())
print(f"전체 결제 유저: {len(paid_users):,}")

# 리텐션 중 수익
revenue_users = retention_users & paid_users
print(f"수익 유저 수: {len(revenue_users):,}")

5단계: 수익
전체 결제 유저: 59,192
수익 유저 수: 503


In [52]:
# 6. 퍼널 지표 계산

print("퍼널 지표")

stages = [len(exposure_users), len(inflow_users), len(participation_users), len(retention_users), len(revenue_users)]

stage_names = ['노출', '유입', '참여', '리텐션', '수익']

print(f"\n{'단계':<10} {'유저 수':>12} {'전환율':>10} {'이탈율':>10}")
print("-" * 50)

for i, (name, count) in enumerate(zip(stage_names, stages)):
    if i == 0:
        print(f"{name:<10} {count:>12,}          -          -")
    else:
        prev = stages[i-1]
        conv = (count / prev * 100) if prev > 0 else 0
        churn = 100 - conv
        print(f"{name:<10} {count:>12,}  {conv:>8.2f}%  {churn:>8.2f}%")

print("\n" + "=" * 80)
print(f"전체 가입 유저 (참고): {len(inflow_all):,}")
if len(exposure_users) > 0:
    final_conv = len(revenue_users) / len(exposure_users) * 100
    print(f"노출→수익 전체 전환율: {final_conv:.2f}%")
print("=" * 80)

퍼널 지표

단계                 유저 수        전환율        이탈율
--------------------------------------------------
노출              626,237          -          -
유입              626,237    100.00%      0.00%
참여               14,991      2.39%     97.61%
리텐션               4,083     27.24%     72.76%
수익                  503     12.32%     87.68%

전체 가입 유저 (참고): 676,978
노출→수익 전체 전환율: 0.08%
